In [ ]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

**Notes:** 

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

The notebook exploits [pyVO](https://pyvo.readthedocs.io/en/latest/), an advanced library  of The [Virtual Observatory](https://ivoa.net/).

[Astroquery](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html) (not used here) is a well-documented, user-friendly alternative.

--------------------------------------------------------

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1
**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [ ]:
# the catalogue name in VizieR
CATALOGUE = "J/MNRAS/390/466"

We first retrieve the catalogue information.

In [ ]:
# the catalogue name in VizieR
CATALOGUE = "J/MNRAS/390/466"
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [ ]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [ ]:
voresource.creators[0]

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [ ]:
tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")

In [ ]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

The actual data can then be accessed using any of the ``access_modes`` of the voresource.

In [ ]:
voresource.access_modes()

The web access is found by following the ``reference_url``

In [ ]:
voresource.reference_url

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [ ]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select * from "{first_table_name}"',
)
tap_records

In [ ]:
tap_records.table.columns

In [ ]:
tables_names

In [ ]:
# get the first table of the catalogue
first_table_name = tables_names[1]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select  * from "{first_table_name}"',
)
tap_records

In [ ]:
tap_records.table.columns

For more information about manipulating multi-order coverages, see [MOCpy documentation](https://cds-astro.github.io/mocpy/), and about using the `ipyaladin` widget, see [its documentation](https://cds-astro.github.io/ipyaladin/).

In [ ]:
import pyvo as vo

# Set up the TAP service
service = vo.dal.TAPService("http://tapvizier.cds.unistra.fr/TAPVizieR/tap")


# Define the query to extract mass (or luminosity) and maximum velocity
query = """
SELECT TOP 10 *
FROM "J/MNRAS/390/466/galaxies"
"""
# Execute the query
result = service.search(query)

# Convert the result to a pandas DataFrame
galaxy_data_10 = result.to_table().to_pandas()
galaxy_data_10.columns

In [ ]:
import pyvo as vo
import numpy as np

R0=14.01E9 # 4D radius of the Universe in lyr
# Set up the TAP service
service = vo.dal.TAPService("http://tapvizier.cds.unistra.fr/TAPVizieR/tap")


# Define the query to extract mass (or luminosity) and maximum velocity
query = """
SELECT
    Name, 
    BMAG,  -- Absolute B magnitude, can be converted to mass or used as a luminosity proxy
    Vmax,   -- Maximum rotation velocity km/s
    Dist   -- Distance Mpc
FROM
    "J/MNRAS/390/466/galaxies"
WHERE
    BMAG IS NOT NULL AND
    Vmax IS NOT NULL
"""

# Execute the query
result = service.search(query)

# Convert the result to a pandas DataFrame
galaxy_data = result.to_table().to_pandas()

# Constants
M_sun = 5.48  # Solar absolute magnitude in the B-band
ML_ratio = 5  # Example M/L ratio, adjust based on your data or literature

# Calculate luminosity relative to the Sun
galaxy_data["Dist_lyr"]= 3_261_563* galaxy_data["Dist"]/R0
galaxy_data['z'] = galaxy_data.Dist_lyr/(1-galaxy_data.Dist_lyr)
galaxy_data['Luminosity'] = 10**(-0.4 * (galaxy_data['BMAG'] - M_sun))

# Estimate mass
galaxy_data['Mass'] = galaxy_data['Luminosity'] * ML_ratio

# Check the first few entries
print(galaxy_data[['Name', 'BMAG', 'Luminosity', 'Mass', 'Vmax', 'Dist_lyr', 'z' ]].head())



In [ ]:
galaxy_data.to_csv('galaxy_data.csv', index=False)

To derive a proxy for mass using distance, you'll need to adjust the luminosity values you've calculated from the B-band magnitudes. The luminosity \( L \) in terms of the absolute magnitude \( M \) can be expressed by the formula:

$ L = 10^{-0.4(M - M_{\text{sun}})} $

where $ M_{\text{sun}} $ is the absolute magnitude of the Sun in the B-band (about 5.48).

However, to calculate luminosity directly from the apparent magnitude and distance, you can use the distance modulus:

$ M = m - 5 \log_{10}(d) + 5 $

where:
-  M is the absolute magnitude,
-  m is the apparent magnitude,
-  d is the distance in parsecs.

If you already have the distance or can calculate it from the redshift (using a cosmological model or approximation if the redshift \( z \) values are available), you can calculate the absolute magnitude and then the luminosity.

Do you have the distances available, or would you need help deriving them from redshift or another method? If you have distances or redshifts in any of your data files, please let me know how you'd like to proceed!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.cosmology import FlatLambdaCDM, z_at_value
import astropy.units as u

# Define the cosmological model
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)

# Define the distance you have
distance = 100 * u.Mpc  # Example distance

# Calculate the redshift corresponding to the luminosity distance
galaxy_data['z'] = [z_at_value(cosmo.luminosity_distance, x*u.Mpc ).value for x in galaxy_data.Dist]
galaxy_data['log1p_z'] = np.log10(galaxy_data.z +1)

# Assuming 'galaxy_data' is the DataFrame with your data
galaxy_data['logMass'] = np.log10( galaxy_data['Mass'])  # Using -log10 because brighter galaxies have smaller magnitude values

# Assuming 'galaxy_data' is the DataFrame with your data
galaxy_data['BMAG'] = galaxy_data['BMAG']  # Using -log10 because brighter galaxies have smaller magnitude values
galaxy_data['logVmax'] = np.log10(galaxy_data['Vmax'])

# PLOT


vmax = galaxy_data["logVmax"]
mass = galaxy_data["logMass"]

# Performing a linear regression to find the Tully-Fisher Coefficient
coefficients = np.polyfit(vmax, mass, 1)
tully_fisher_coefficient = coefficients[0]

# Using the fitted line to calculate predicted values
predicted_mass = np.polyval(coefficients, vmax)

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(vmax, mass, c='blue', edgecolor='black', label='Data')
plt.plot(vmax, predicted_mass, 'r-', label=f'Fit Line: slope = {tully_fisher_coefficient:.2f}')
plt.title(f'Log10 Mass vs. log10 of Maximum Velocity\nTully-Fisher Coefficient: {tully_fisher_coefficient:.2f}')
plt.xlabel('log10(Vmax [km/s])')
plt.ylabel('log10 Mass')
plt.text(2.3, 9.1, 'vizier_jmnras390466', fontsize=12, color='red')
plt.grid(True)
plt.legend()
plt.savefig("./Drawing_For_Publications/GHasp_TullyFisher_M_vs_Vmax.png")
plt.show()

tully_fisher_coefficient

In [ ]:
print(galaxy_data.iloc[0:2])

In [ ]:
from scipy.stats import linregress

# Example: Fitting logMass vs. log(1+z)
slopeLogMass, intercept, r_value, p_value, std_err = linregress(galaxy_data['log1p_z'], galaxy_data['logMass'])
print(f"Slope: {slope}, Intercept: {intercept}")


In [ ]:
slopeVMax, intercept, r_value, p_value, std_err = linregress(galaxy_data['log1p_z'], galaxy_data['logVmax'])
print(f"Slope: {slope}, Intercept: {intercept}")

In [ ]:
slopeBMAG, intercept, r_value, p_value, std_err = linregress(galaxy_data['log1p_z'], galaxy_data['BMAG'])
print(f"Slope: {slope}, Intercept: {intercept}")

In [ ]:
slopeLogMass/slopeVMax , slopeLogMass/slopeBMAG

In [ ]:
# Assuming galaxy_data is already loaded with logMass and logVmax

# Fit and plot directly
coefficients = np.polyfit( galaxy_data['log1p_z'], galaxy_data['logMass'], 1)
predicted_mass = np.polyval(coefficients, galaxy_data['log1p_z'])

plt.figure(figsize=(10, 6))
plt.scatter(galaxy_data['log1p_z'], galaxy_data['logMass'], label='Data Points')
plt.plot(galaxy_data['log1p_z'], predicted_mass, 'r-', label=f'Fit Line: slope = {coefficients[0]:.2f}')
plt.xlabel('log10(1+z)')
plt.ylabel('log10(Mass)')
plt.title('Re-Evaluated Tully-Fisher Relationship')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Assuming galaxy_data is already loaded with logMass and logVmax

# Fit and plot directly
coefficients = np.polyfit(galaxy_data['log1p_z'], galaxy_data['logVmax'], 1)
predicted_Vmax = np.polyval(coefficients, galaxy_data['log1p_z'])

plt.figure(figsize=(10, 6))
plt.scatter(galaxy_data['log1p_z'], galaxy_data['logVmax'], label='Data Points')
plt.plot(galaxy_data['log1p_z'], predicted_Vmax, 'r-', label=f'Fit Line: slope = {coefficients[0]:.2f}')
plt.xlabel('log10(1+z)')
plt.ylabel('log10(VMax)')
plt.title('Re-Evaluated Tully-Fisher Relationship')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from astroquery.utils.tap.core import TapPlus

# Connect to VizieR TAP service
vizier_tap_url = "http://tapvizier.u-strasbg.fr/TAPVizieR/tap"
tap_service = TapPlus(url=vizier_tap_url)

# Construct an ADQL query
query = """
SELECT TOP 10
    *
FROM
    "V/147/sdss12"
WHERE
    1=1
"""

# Execute the query using query_adql or query_job
job = tap_service.launch_job_async(query)
result = job.get_results()

# Show the results
print(result)


In [ ]:
# Simbad
import requests

def query_simbad(object_name):
    url = "http://simbad.u-strasbg.fr/simbad/sim-script"
    script = f"format object form1\nquery id {object_name}\noutput console=off script=off\nvotable {{main_id,coordinates,otypes,velocity,redshift,flux(B),dim(maj_axis),dim(min_axis)}}"
    response = requests.post(url, data={'script': script})
    
    if response.status_code == 200:
        return response.text
    else:
        return "Failed to retrieve data"

# Example usage
object_name = "UGC 763"  # Replace with your object of interest
result = query_simbad(object_name)
print(result)


In [ ]:
from astropy.io.votable import parse_single_table

# Example function to load and parse VOTable
def load_votable(file_path):
    table = parse_single_table(file_path).to_table()
    return table

# Assuming 'data.xml' is your downloaded VOTable file from SIMBAD
data_table = load_votable('data.xml')
print(data_table)

